# MasterFunctions - Signal Rates

This notebook contains all function definitions required to generate the signal plots for IceCube

In [1]:
import io, os, sys, types

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
    
sys.meta_path.append(NotebookFinder())

# print ('Complete')

In [2]:
from MasterFunctions_Capture import *
from MasterFunctions_Annihilation import *

# print ('Notebooks Imported')

importing Jupyter notebook from MasterFunctions_Capture.ipynb
importing Jupyter notebook from MasterFunctions_ModelData.ipynb
------ MasterFunctions_ModelData Imported ------
------ MasterFunctions_Capture Loaded ------
importing Jupyter notebook from MasterFunctions_Annihilation.ipynb
------ MasterFunctions_Annihilation Loaded ------


------------------

# Signal Rate Functions

## Equilibrium Timescale

In [3]:
def tau(CCap,CAnn): # CCap and CAnn are both in sec^-1
    function = 1./(np.sqrt(CCap*CAnn))
    
    return function

## Gamma Annihilation

In [55]:
def gammaAnn(Ccap, Cann):
    Tau = tau(Ccap, Cann)
    EQRatio = tauCross/Tau
    function = (0.5) * Ccap * ((np.tanh(EQRatio))**2)
    
    return function

## Decay Width

I don't think I actually call these anywhere in my code

In [56]:
def decayWidth1(mA, mf, epsilon, alpha):
    Nc = 1
    qf = 1
    function = Nc * epsilon**2 * qf * alpha * (mA**2 + 2*mf**2) * \
        np.sqrt(1-(4*mf**2/mA**2)) *(1/(3*mA))
    return function

def decayWidth2(mA, epsilon, alpha):
    function= alpha*epsilon**2*mA/3
    return function

# def decayWidthJordan(mA, mf, epsilon, alpha):
#     Nc = 1
#     qf = 1
#     function = Nc * epsilon**2 * qf * alpha * (mA**2 + 2*mf**2) * \
#         np.sqrt(1-(4*mf**2/mA**2)) *(1/(3*mA))
#     return function

# print (decayWidth1(1, 0.01, 10**-8, 1./137))
# print (decayWidth2(1,10**-8,1./137))

## Decay Length (Eqn 3 Sun Paper)

In [57]:
######################################
# This is EQN 3 in the Sun paper
######################################

# def decayLength1(m_X, m_A, epsilon, BR): # Returns cm
#     RDot = 7e10 # cm, radius of sun
#     function = RDot * BR * (1.1e-9/epsilon)**2 * (m_X/m_A) * (1./1000) * (0.1/m_A)
#     return function

######################################
# This is EQN 4 in the Earth paper
######################################
def decayLength(m_X, m_A, epsilon, BR):
    '''
    decayLength(m_X, m_A, epsilon, BR) returns the length of travel in cm, since the radius of
    the Earth, RCross is given in cm.
    '''
    function = RCross * BR * (3.6e-9/epsilon)**2 * (m_X/m_A) * (1./1000) * (1./m_A)
    return function


## Deacay Parameter $\varepsilon$ (Eqn 31)

In [58]:
def epsilonDecay(decayLength, effectiveDepth = 10**5): # Effective depth = 1 km
    '''
    epsilonDecay(decayLength, effectiveDepth = 10**5) For this function, the effectiveDepth parameter
    is set to the default value for the IceCube Neutrino Observatory of 1 km.
    '''
    arg1 = RCross
    arg2 = RCross + effectiveDepth
    
    function = np.exp(-arg1/decayLength) - np.exp(-arg2/decayLength)
    return function


3492979199999.9995
2.86233340363e-08


## IceCube Signal Rates

In [59]:
# Live time T must be in Seconds
def iceCubeSignal(gammaAnn, epsilonDecay, T, Aeff = 10**10):
    function = 2 * gammaAnn * (Aeff/ (4*np.pi*RCross**2) ) * epsilonDecay * T
    return function


## Sanity Check

In [6]:
# m_X = 1000.
# m_A = 0.5
# epsilon = 10**-8
# alpha = 1./137
# alpha_X = alphaTherm(m_X, m_A)

# BR = 1
# LiveTime = yr2s(10)

# print (RCross)
# print (LiveTime)
# print ('Complete')

673800000.0
315360000.0
Complete


In [13]:
# kappa = kappa_0(m_X, alpha)
# Capture = CCapQuick(m_X, m_A, epsilon, alpha_X, kappa)

# Sommerfeld = thermAvgSommerfeld(m_X, m_A, alpha_X)
# Annihilation = CAnnCalc(m_X, m_A, alpha_X, Sommerfeld) 

# # Tau = tau(Capture, Annihilation)

# # EQRatio = tauCross/Tau

# print ('Complete')

Complete


In [21]:
# GammaAnn = gammaAnn(Capture, Annihilation)
# print (GammaAnn)
# # DecayLength = decayLength2(m_X, m_A, epsilon, BR)
# # DecayLength = decayLengthJordan(m_X, m_A, epsilon, BR)

# # EpsilonDecay = epsilonDecay(DecayLength)
# # Signal = iceCubeSignal(GammaAnn, EpsilonDecay, LiveTime)

873484806.864


In [23]:
# D = 10**5

# print (.1/function)

-0.0


C:\Users\Adam Green\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [84]:
# print ('Capture      : {0}'.format(Capture))
# print ('Annihilation : {0}'.format(Annihilation))
# print ('Tau          : {0}'.format(Tau))
# print ('EQ Ratio     : {0}'.format(EQRatio))
# print ('GammaAnn     : {0}'.format(GammaAnn))
# print ('Decay Length : {0}'.format(DecayLength))
# print ('Epsilon Decay: {0}'.format(EpsilonDecay))
# print ('Signal       : {0}'.format(Signal))

Capture      : 109185621.33035141
Annihilation : 1.6600111103365965e-46
Tau          : 8.706969213329477e+16
EQ Ratio     : 52.34107727822834
GammaAnn     : 19922.528902253896
Decay Length : 11100000000.0
Epsilon Decay: 8.478367084507887e-06
Signal       : 0.1867331429333801


## Sanity Checks

For the following parameters:

- $m_X = 1000$ GeV

- $m_A = 1.0$ GeV

- $\varepsilon = 10^{-8}$

we know for certain:
- $C_{Cap} \sim 1.09185621\times 10^8$

- $C_{Ann} \sim 1.66001111034\times 10^{-46}$ (with Sommerfeld)

- $\frac{\tau}{\tau_{\oplus}} \mathcal{O}(10^1 - 10^2)$

- $N_{sig} \sim \mathcal{O}(10^2 - 10^3)$



In [75]:
# testm_X = 1000.
# testm_A = 1.0
# testEpsilon = 10**-8
# alpha_X = alphaTherm(testm_X, testm_A)
# LiveTime = yr2s(10)
# BranchingRatio = 1

# # # Capture Functions
# Kappa0 = kappa_0(testm_X, 1./137)
# Capture = CCapQuick(testm_X, testm_A, testEpsilon, alpha_X, Kappa0)

# # # Annihilation Functions
# Sommerfeld = thermAvgSommerfeld(testm_X, testm_A, alpha_X)
# Annihilation = CAnn(testm_X, testm_A, alpha_X, Sommerfeld)

# # # Equilibrium Time
# EQTime = tau(Capture, Annihilation)
# EQRatio = EQTime/tauCross

# # # Signal Rates
# GammaAnn = gammaAnnCalc(Capture, Annihilation)
# # DecayLength = decayLength2(testm_X, testm_A, testEpsilon, BranchingRatio)
# DecayLength = decayLengthJordan(testm_X, testm_A, testEpsilon, BranchingRatio)
# EpsilonDecay = epsilonDecay(DecayLength)
# Signal = iceCubeSignal(GammaAnn, EpsilonDecay, LiveTime)


# print ('Complete')

In [77]:
# print ('Capture       : {0}'.format(Capture))
# print ('Annihilation  : {0}'.format(Annihilation))
# print ('Tau:          : {0}'.format(EQTime))
# print ('EQ Ratio      : {0}'.format(EQRatio))
# print ('GammaAnn      : {0}'.format(GammaAnn))
# print ('DecayLength   : {0}'.format(DecayLength))
# print ('RCross        : {0}'.format(RCross))
# print ('Epsilon Decay : {0}'.format(EpsilonDecay))
# print ('Signal        : {0}'.format(Signal))

# print ('Complete')

Capture       : 109185621.33035141
Annihilation  : 1.6600111103365965e-46
Tau:          : 7.42782695870794e+18
EQ Ratio      : 52.34107727822834
GammaAnn      : 19922.528902253896
DecayLength   : 11100000000.0
RCross        : 673800000.0
Epsilon Decay : 8.478367084507887e-06
Signal        : 0.1867331429333801


In [ ]:
print ('------ MasterFunctions_SignalRates Imported ------')